In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

%matplotlib inline
title = 'PPD'
path = '../sources/data/PPD-First-Round-Data'
icy = 'target'

def Del_string(xstr):
    xstrc = xstr.strip().strip(u'市').strip(u'省')
    if(xstrc == ''):
        xstrc = np.nan
    return(xstrc)

## Initiate Data

Read data from orginal data files, save them into database which is easier to reload

### 读取初赛轮（First Round）中，TrainSet & TestSet中的Master数据
#### TrainSet (30000, 227)  |  TestSet (19999, 226)

In [5]:
# 读取初赛trainset和testset master
# 备注:原始数据中某些属性的取值已经是-1了，但从逻辑上讲，仍应表示缺失值
par_csv = dict(index_col = 0, encoding = 'GB18030', parse_dates = ["ListingInfo"], na_values = [-1], 
               converters = dict(zip(*[["UserInfo_{}".format(i) for i in [2,4,7,8,9,19,20]], [Del_string]*7])))
print par_csv

dat_master = pd.read_csv('{}/TrainingSet/PPD_FR_TrainSet_Master.csv'.format(path), **par_csv)
dav_master = pd.read_csv('{}/TestSet/PPD_FR_TestSet_Master.csv'.format(path), **par_csv)
print dat_master.shape
print dav_master.shape
da = pd.concat([dat_master, dav_master])

{'na_values': [-1], 'parse_dates': ['ListingInfo'], 'converters': {'UserInfo_2': <function Del_string at 0x119d82d70>, 'UserInfo_7': <function Del_string at 0x119d82d70>, 'UserInfo_4': <function Del_string at 0x119d82d70>, 'UserInfo_19': <function Del_string at 0x119d82d70>, 'UserInfo_8': <function Del_string at 0x119d82d70>, 'UserInfo_9': <function Del_string at 0x119d82d70>, 'UserInfo_20': <function Del_string at 0x119d82d70>}, 'index_col': 0, 'encoding': 'GB18030'}
(30000, 227)
(19999, 226)


In [7]:
np.save('../sources/deal/PPD-Data-Saving/{}_irt.npy'.format(title), list(dat_master.index))
np.save('../sources/deal/PPD-Data-Saving/{}_irv.npy'.format(title), list(dav_master.index))
da.to_hdf('../sources/deal/PPD-Data-Saving/{}_da.h5'.format(title), key = 'da', complib = 'zlib', complevel = 1, mode = 'w')

# dat_master.head()
# dav_master.head()

/Users/fanghan/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->axis0] [items->None]

  f(store)
/Users/fanghan/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block0_items] [items->None]

  f(store)
/Users/fanghan/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_items] [items->None]

  f(store)
/Users/fanghan/anaconda/lib/python2.7/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map dire

### 读取First Round中，TrainSet & TestSet中的loginfo & userupdate数据

In [8]:
dat_loginfo = pd.read_csv('{}/TrainingSet/PPD_FR_TrainSet_LogInfo.csv'.format(path), parse_dates = ['Listinginfo1','LogInfo3'])
dav_loginfo = pd.read_csv('{}/TestSet/PPD_FR_TestSet_LogInfo.csv'.format(path), parse_dates = ['Listinginfo1','LogInfo3'])
dah_loginfo = pd.concat([dat_loginfo, dav_loginfo])
print dah_loginfo.head()
dah1 = dah_loginfo.assign(Id=dah_loginfo['Idx'], \
                          Time = (dah_loginfo['LogInfo3']-dah_loginfo['Listinginfo1']).astype('timedelta64[D]'))\
           .set_index(['Id','Time']).drop(['Idx','Listinginfo1','LogInfo3'], axis=1).sort_index()
print '\n'
print dah1.head()

dat_uu = pd.read_csv('{}/TrainingSet/PPD_FR_TrainSet_Userupdate.csv'.format(path), \
                     parse_dates = ['ListingInfo1','UserupdateInfo2'],\
                     converters = {'UserupdateInfo1': lambda x: x.lower()})
dav_uu = pd.read_csv('{}/TestSet/PPD_FR_TestSet_Userupdate.csv'.format(path), \
                     parse_dates = ['ListingInfo1','UserupdateInfo2'],\
                     converters = {'UserupdateInfo1': lambda x: x.lower()})
dah_uu = pd.concat([dat_uu, dav_uu])
print '\n'
print dah_uu.head()
dah2 = dah_uu.assign(Id=dah_uu['Idx'], \
              Time=(dah_uu['UserupdateInfo2']-dah_uu['ListingInfo1']).astype('timedelta64[D]'))\
      .set_index(['Id','Time']).drop(['Idx','ListingInfo1','UserupdateInfo2'], axis=1).sort_index()
print '\n'
print dah2.head()

     Idx Listinginfo1  LogInfo1  LogInfo2   LogInfo3
0  10001   2014-03-05       107         6 2014-02-20
1  10001   2014-03-05       107         6 2014-02-23
2  10001   2014-03-05       107         6 2014-02-24
3  10001   2014-03-05       107         6 2014-02-25
4  10001   2014-03-05       107         6 2014-02-27


          LogInfo1  LogInfo2
Id Time                     
3  -67.0         1         0
   -67.0         1         1
   -67.0         1         4
   -67.0         2         1
   -67.0         4         1


     Idx ListingInfo1    UserupdateInfo1 UserupdateInfo2
0  10001   2014-03-05       _educationid      2014-02-20
1  10001   2014-03-05         _hasbuycar      2014-02-20
2  10001   2014-03-05    _lastupdatedate      2014-02-20
3  10001   2014-03-05  _marriagestatusid      2014-02-20
4  10001   2014-03-05       _mobilephone      2014-02-20


            UserupdateInfo1
Id Time                    
3  -67.0       _educationid
   -67.0         _hasbuycar
   -67.0          _

In [9]:
dah1.to_hdf('../sources/deal/PPD-Data-Saving/{}_dah1.h5'.format(title), key = 'dah', complib = 'zlib', complevel = 1, mode = 'w')
dah2.to_hdf('../sources/deal/PPD-Data-Saving/{}_dah2.h5'.format(title), key = 'dah', complib = 'zlib', complevel = 1, mode = 'w')

## Reload Data
Load data from database, then concatenate and summerize data

In [10]:
da = pd.read_hdf('../sources/deal/PPD-Data-Saving/{}_da.h5'.format(title), key = 'da')
dah1 = pd.read_hdf('../sources/deal/PPD-Data-Saving/{}_dah1.h5'.format(title), key = 'dah')
dah2 = pd.read_hdf('../sources/deal/PPD-Data-Saving/{}_dah2.h5'.format(title), key = 'dah')
irt = np.load('../sources/deal/PPD-Data-Saving/{}_irt.npy'.format(title))
irv = np.load('../sources/deal/PPD-Data-Saving/{}_irv.npy'.format(title))
print da.shape
print dah1.shape
print dah2.shape

(49999, 227)
(966431, 2)
(621295, 1)


### Preprocessing & Concatenate Historical Data with Master Data

In [130]:
def Preprocess_historicalData(dah, iccat, name, da):
    grp1 = dah.reset_index()[["Id", "Time"]].groupby("Id")
    grp2 = dah.groupby(level = ["Id", "Time"]).first().reset_index()[["Id", "Time"]].groupby("Id")
    dahc1 = pd.concat([grp1.first(), grp1.count()], axis = 1, ignore_index = True).rename(columns = {0:"FirstTime", 1:"Count"})
    dahc1 = dahc1.assign(DayFreq = grp2.count()["Time"]/(1-dahc1["FirstTime"])).loc[da.index]
    dahc2 = dah.reset_index().groupby(["Id"]+iccat).count().unstack(iccat)["Time"].loc[da.index]
    dahc = pd.concat([dahc1, pd.DataFrame({"Cats": dahc2.notnull().sum(axis = 1)}), dahc2], axis = 1).fillna(0)
    dahc.columns = ["{}_{}".format(name, x) for x in dahc.columns]
    return(dahc)

dah1c = Preprocess_historicalData(dah1, list(dah1.columns), 'Log', da)
dah2c = Preprocess_historicalData(dah2, list(dah2.columns), 'Userupdate', da)
# print dah1c.head()
# print dah2c.head()
da = pd.concat([da, dah1c, dah2c], axis = 1)
da.shape

(49999, 352)